In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img
from PIL import Image

In [ ]:
def read_image(path):
    # Reads an image from the determined path
    # The image has to have a bit-depth of 8 (each pixel's value is in the 0-255 range)
    img_to_recognize = img.imread(path)
    img_to_recognize = Image.fromarray(np.uint8(img_to_recognize))
    img_to_recognize = img_to_recognize.resize((150, 150)) # Resizes the image to the appropriate size
    img_to_recognize = img_to_recognize.convert('L') # Converts it to grayscale
    img_to_recognize = np.array(img_to_recognize) # Converts it into an array
    return img_to_recognize.flatten()/255 # Flattens and normalizes the data

In [ ]:
def display_image_ind(matrix, index):
    # Displays the image present in the row_{index} of the specified matrix
    resized = np.resize(matrix[index], (150, 150))
    fig = plt.imshow(resized, cmap="gray")
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
    fig


def display_image(matrix):
    # Displays the image present in the row_{index} of the specified matrix
    resized = np.resize(matrix, (150, 150))
    fig = plt.imshow(resized, cmap="gray")
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
    fig



def plot_portraits(images, titles, h, w, n_row, n_col):
    plt.figure(figsize=(2.2 * n_col, 2.2 * n_row))
    plt.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.20)
    for i in range(n_row * n_col):
        plt.subplot(n_row, n_col, i + 1)
        plt.imshow(images[i].reshape((h, w)), cmap=plt.cm.gray)
        plt.title(titles[i])
        plt.xticks(())
        plt.yticks(())

In [ ]:
images = [] # List that will house all individual arrays of images, it will become a 2D array later

for file in os.listdir('faces'):
    images.append(read_image(os.path.join('faces', file)))


In [ ]:
celebrity_photos=os.listdir('faces')
celebrity_names=[name[:name.find('0')-1].replace("_", " ") for name in celebrity_photos]


In [ ]:
image_matrix = np.row_stack(tuple(images))


In [ ]:
def pca(X):
    # X is the data matrix, n_pc is the number of requested eigenvectors/components
    mean = np.mean(X, axis=0)
    centered_data = X-mean
    U, S, Vh = np.linalg.svd(centered_data, full_matrices=False)
    
    return Vh, mean, centered_data, S**2


In [ ]:
Vh, average_matrix, subtracted, eigenvalues = pca(image_matrix)
eigenfaces = Vh.reshape((434, 150, 150))

In [ ]:
percent_eigenvalues = [eigenvalue/np.sum(eigenvalues) for eigenvalue in eigenvalues]
count = 0
total_var = 0

for eigenvalue in percent_eigenvalues:
    total_var += eigenvalue
    count += 1
    if total_var > 0.95:
        break

print("Count:", count, "\nTotal Variance:", total_var)

In [ ]:
display_image(average_matrix)

plt.savefig('average face.png', bbox_inches='tight')

In [ ]:
# eigenface_titles = [f"eigenface {434-i}" for i in range(eigenfaces.shape[0])]
# plot_portraits(eigenfaces[::-1], eigenface_titles, 150, 150, 2, 5) 
# plt.savefig('lowest eigenfaces.png', bbox_inches='tight')

eigenface_titles = [f"eigenface {i+91}" for i in range(eigenfaces.shape[0])]
plot_portraits(eigenfaces[90:105,:], eigenface_titles, 150, 150, 2, 5) 
plt.savefig('medium eigenfaces.png', bbox_inches='tight')

In [ ]:
# display_image_ind(image_matrix, 307)


In [ ]:
display_image_ind(image_matrix, 142)
plt.savefig('original face.png', bbox_inches='tight')

In [ ]:
def reconstruction(centered_data, eigenfaces, average, h, w, image_index):
    weights = np.dot(centered_data, eigenfaces.T) # Gets the weight significance of each eigenface
    weighted_vectors = np.dot(weights[image_index, :], eigenfaces) # Multiplies each eigenface by its weight
    recovered_image = (average + weighted_vectors).reshape(h, w) # Adds each weighted eigenface to the average face
    return recovered_image

display_image(reconstruction(subtracted, Vh[:300,:], average_matrix, 150, 150, 142))
plt.savefig('reconstructed full face.png', bbox_inches='tight')

In [ ]:
def reconstruction_outsider(eigenfaces, average, h, w, path):
    outsider = read_image(path) # Reads the image
    outsider = outsider - average # Centralizes the data
    weights = np.dot(outsider, eigenfaces.T) # Gets the weight significance of each eigenface
    weighted_vectors = np.dot(weights, eigenfaces) # Multiplies each eigenface by its weight
    recovered_image = (average + weighted_vectors).reshape(h, w) # Adds each weighted eigenface to the average face
    return recovered_image

In [ ]:
def recognize(path, eig_num, face_limit, person_limit):
    # Path is the file path, eig_num is the amount of wanted eigenfaces on the facespace
    img_to_recognize = read_image(path)
    subtracted_matrix_rec = img_to_recognize - average_matrix # Centralizes the inputted image
    subtracted_matrix_rec = subtracted_matrix_rec.flatten()

    eigenfaces_matrix = Vh[:eig_num,:] # Gets the requested amount of eigenfaces

    weight = subtracted_matrix_rec @ eigenfaces_matrix.T # Gets the eigenface weights

    projection = eigenfaces_matrix.T @ weight # Gets the projection of the image on the facespace

    proj_error = np.linalg.norm(subtracted_matrix_rec - projection) # Gets the projection error

    if proj_error > face_limit: # Checks if the error is higher than the set limit
        return "Not a face!"
    

    original_faces_weights = eigenfaces_matrix @ subtracted.T # Gets the eigenface weights of each original face


    dist_in_space = []
    # Checks the distance between the weights 
    # of the unknown face and every original face
    for i in range(len(subtracted[:,0])):
        dist = np.linalg.norm(original_faces_weights[:,i] - weight)
        dist_in_space.append(dist)
    
    dist_in_space = np.array(dist_in_space)

    face_error = dist_in_space.min() # Gets the lowest distance

    # Trying to get the right face (range of valid error still to be determined)
    guess_index = np.argmin(dist_in_space)
    
    celebrity_name = celebrity_names[guess_index]

    display_image(image_matrix[guess_index])
    return proj_error*255, face_error, celebrity_name
    

In [ ]:
path_recon = os.path.join('objects', 'Keanu_Reeves_0008.pgm')
display_image(read_image(path_recon))

In [ ]:
display_image(reconstruction_outsider(Vh, average_matrix, 150, 150, path_recon))

In [ ]:
recognize(path_recon, 106, 3000, 30)

In [ ]:
# george = os.path.join('lfwcrop_grey', 'faces')

# numbers = []
# for number in range(9, 531):
#     str_num = "0"*(4-len(str(number))) + str(number)
#     numbers.append(str_num)

In [ ]:
# george_list = []
# for number in numbers:
#     george_dict = {}
#     george_dict["Projection Error"], george_dict["Face Error"], george_dict["Celebrity Name"] = recognize2(george, 'George_W_Bush_', number, 100, '.pgm')
#     george_list.append(george_dict)

In [ ]:
# pred_names = []
# for dic in george_list:
#     pred_names.append(dic['Celebrity Name'])

# pred_names = dict(collections.Counter(pred_names))

# highest = 0 
# for value in pred_names.values():
#     if value > highest:
#         highest = value

# pred_names['George W Bush'], highest

In [ ]:
plt.plot(eigenvalues)
plt.xticks(range(0,200,20))
plt.xlim(-1, 100)
plt.show()